In [1]:
from imports import *
from PreProcessor import *
import os
import numpy as np

In [2]:

CodeTokens = set()
UCTokens = set()
UC_documents = list()
code_documents = list()
TotalTokens=set()
entropy_uc = []
entropy_code = []
variance_uc = {}
variance_code = {}
SCQ_uc={}
SCQ_code={}
term_co_occurrences_uc = {}
PMI_uc = {}
term_co_occurrences_code= {}
PMI_code = {}


In [3]:
# Q, the set of query terms; q, a term in the query; D, the set of documents in the collection;
# Dt, the set of documents containing term t
# d, a document in the document collection D;
# tf(t, D), the frequency of term t in all docs;
# tf(t, d), the frequency of term t in d;
# tf(t, Q), the frequency of term t in the query;
# sim(di, dj), the cosine similarity between the vector-space representations of di and dj
# idf(t) = log( |D|/|Dt|).
# ictf(t) = log( |D|/tf(t,D) ).


In [4]:
for filename in os.listdir("./UC"):
    filepath = os.path.join("./UC", filename)
    tokens = PreProcessor().UCPreProcessor(filepath)
    UC_documents.append(tokens)
    UCTokens.update(tokens.split())


for filename in os.listdir("./CC"):
    filepath = os.path.join("./CC", filename)
    tokens = PreProcessor().CodePreProcessor(filepath)
    code_documents.append(tokens)
    CodeTokens.update(tokens.split())

TotalTokens = CodeTokens.union(UCTokens)


In [5]:
vectorizer_uc = TfidfVectorizer(vocabulary=TotalTokens)
tf_matrix_uc=vectorizer_uc.fit_transform(UC_documents)
idf_uc = vectorizer_uc.idf_
feature_names = vectorizer_uc.get_feature_names_out() 
idf_uc_dict = {feature_names[i]: idf_uc[i] for i in range(len(feature_names))}  #this is the idf dictionary for the whole vocab that i acsess later doing whatever

vectorizer_code = TfidfVectorizer(vocabulary=TotalTokens)
tf_matrix_code=vectorizer_code.fit_transform(code_documents)
idf_code = vectorizer_code.idf_
feature_names = vectorizer_code.get_feature_names_out() 
idf_code_dict = {feature_names[i]: idf_code[i] for i in range(len(feature_names))}


# IDF 


In [6]:
idf_uc_q=[]
for Querey in UC_documents:
    tokens = Querey.split()
    idf_q=[]
    for token in tokens:
        idf_q.append(idf_uc_dict[token])
    idf_uc_q.append(idf_q)

idf_code_q=[]
for Querey in code_documents:
    tokens = Querey.split()
    idf_q=[]
    for token in tokens:
        idf_q.append(idf_code_dict[token])
    idf_code_q.append(idf_q)



def AvgIDF(idf_values):
    avgidf=[]
    for query in idf_values:
        avgidf.append(sum(query)/len(idf_values))    # |G| is len(idf_values)
    return avgidf

def MaxIDF(idf_values):
    maxidf=[]
    for query in idf_values:
        maxidf.append(max(query))
    return maxidf


def DevIDF(idf_values):
    devidf=[]
    avgidf=AvgIDF(idf_values)
    for query_idf in idf_values:
        deviation = np.sqrt(sum([abs(idf - avg) for idf, avg in zip(query_idf, avgidf)]) / len(idf_values))
        devidf.append(deviation)
    return devidf
       


# ICTF

In [7]:


#ictf(t) = log( |D|/tf(t,D) ).
ictf_uc_q=[]
for Querey in UC_documents:
    tokens = Querey.split()
    ictf_q=[]
    for token in tokens:
        ictf_q.append(np.log(len(UC_documents)/Querey.count(token)))
    ictf_uc_q.append(ictf_q)

ictf_code_q=[]
for Querey in code_documents:
    tokens = Querey.split()
    ictf_q=[]
    for token in tokens:
        ictf_q.append(np.log(len(code_documents)/Querey.count(token)))
    ictf_code_q.append(ictf_q)



def AvgICTF(ictf_values):
    avgictf=[]
    for query in ictf_values:
        avgictf.append(sum(query) / len(query))
    return avgictf

def MaxICTF(ictf_values):
    maxictf=[]
    for query in ictf_values:
        maxictf.append(max(query))
    return maxictf

def DevICTF(ictf_values):
    devictf=[]
    avgictf=AvgICTF(ictf_values)
    for query_ictf in ictf_values:
        deviation = np.sqrt(sum([abs(ictf - avg)for ictf ,avg in zip(query_ictf,avgictf)]) / len(ictf_values))
        devictf.append(deviation) 
    return devictf






# entropy

In [8]:
idf_uc_q=[]
for Querey in UC_documents:
    tokens = Querey.split()
    idf_q=[]
    for token in tokens:
        idf_q.append(idf_uc_dict[token])
    idf_uc_q.append(idf_q)

# Q, the set of query terms; q, a term in the query; D, the set of documents in the collection;
# Dt, the set of documents containing term t
# d, a document in the document collection D;
# tf(t, D), the frequency of term t in all docs;
# tf(t, d), the frequency of term t in d;                @isn't in my case the querey is the document ?????
# tf(t, Q), the frequency of term t in the query;
# sim(di, dj), the cosine similarity between the vector-space representations of di and dj
# idf(t) = log( |D|/|Dt|).
# ictf(t) = log( |D|/tf(t,D) ).
#entropy(t) = sumd∈Dt(tf(t,d)/tf(t,D) · log|D|tf(t,d)/tf(t,D)  
entropy_uc = [] 

df_uc = np.sum(tf_matrix_uc > 0, axis=0).A1
total_documents_uc = len(UC_documents)
ictf_uc = np.log(total_documents_uc / (df_uc + 1))

for term_index, term in enumerate(vectorizer_uc.get_feature_names_out()):
    term_entropy = []
    for doc in UC_documents:
        tf_term_doc = doc.count(term) + 1 
        tf_term_collection = df_uc[term_index] + 1 
        term_entropy.append((tf_term_doc / tf_term_collection) * np.log((tf_term_doc / tf_term_collection) + 1))
    entropy_uc.append(term_entropy)

entropy_code= []

df_code = np.sum(tf_matrix_code > 0, axis=0).A1
total_documents_code = len(code_documents)
ictf_code = np.log(total_documents_code / (df_code + 1))

for term_index, term in enumerate(vectorizer_code.get_feature_names_out()):
    term_entropy = []
    for doc in code_documents:
        tf_term_doc = doc.count(term) + 1 
        tf_term_collection = df_code[term_index] + 1 
        term_entropy.append((tf_term_doc / tf_term_collection) * np.log((tf_term_doc / tf_term_collection) + 1))
    entropy_code.append(term_entropy)

def AvgEntropy(entropy_values):
    avgentropy=[]
    for query in entropy_values:
        avgentropy.append(sum(query) / len(entropy_values))
    return avgentropy
    
def MaxEntropy(entropy_values):
    maxentropy=[]
    for query in entropy_values:
        maxentropy.append(max(query))
    return maxentropy
def MedEntropy(entropy_values):
    medentropy=[]
    for query in entropy_values:
        medentropy.append(np.median(query))
    return medentropy

def DevEntropy(entropy_values):
    deventropy=[]
    avgentropy = AvgEntropy(entropy_values)
    for query_entropy in entropy_values:
        deviation = np.sqrt(sum([abs(entropy - avgentropy[i]) for i, entropy in enumerate(query_entropy)]) / len(entropy_values))
        deventropy.append(deviation)
    return deventropy

# VAR

In [9]:
variance_uc = [] 

for query in UC_documents:
    tokens = query.split()
    query_variances = []  
    for term in tokens:
        term_weights = []
        for doc in UC_documents:
            tf_term_doc = doc.count(term)
            weight_term_doc = (1 / len(doc)) * np.log(1 + tf_term_doc) * idf_uc_dict[term]  #
            term_weights.append(weight_term_doc)

        avg_weight_term = np.mean(term_weights)
        variance_term = np.mean([(weight - avg_weight_term) ** 2 for weight in term_weights])
        query_variances.append(variance_term) 
    variance_uc.append(query_variances)  

variance_code = []

for query in code_documents:
    tokens = query.split()
    query_variances = []  
    for term in tokens:
        term_weights = []
        for doc in code_documents:
            tf_term_doc = doc.count(term)
            weight_term_doc = (1 / len(doc)) * np.log(1 + tf_term_doc) * idf_code_dict[term]  #
            term_weights.append(weight_term_doc)

        avg_weight_term = np.mean(term_weights)
        variance_term = np.mean([(weight - avg_weight_term) ** 2 for weight in term_weights])
        query_variances.append(variance_term) 
    variance_code.append(query_variances)



def AvgVariance(variance_values):
    avgvariance=[]
    for query in variance_values:
        avgvariance.append(sum(query) / len(variance_values))
    return avgvariance

def MaxVariance(variance_values):
    maxvariance=[]
    for query in variance_values:
        maxvariance.append(max(query))
    return maxvariance

def SumVariance(variance_values):
    sumvariance=[]
    for query in variance_values:
        sumvariance.append(sum(query))
    return sumvariance


# SCQ

In [10]:
SCQ_uc = []  

for document in UC_documents:
    document_SCQ = []

    for term_index, term in enumerate(vectorizer_uc.get_feature_names_out()):
        tf_term_doc = document.count(term) + 1 
        document_SCQ.append((1 + np.log(tf_term_doc)) * idf_uc[term_index])
    SCQ_uc.append(document_SCQ)

SCQ_code = []

for document in code_documents:
    document_SCQ = []

    for term_index, term in enumerate(vectorizer_code.get_feature_names_out()):
        tf_term_doc = document.count(term) + 1 
        document_SCQ.append((1 + np.log(tf_term_doc)) * idf_code[term_index])
    SCQ_code.append(document_SCQ)


def AvgSCQ(SCQ_values):
    avgscq=[]
    for query in SCQ_values:
        avgscq.append(sum(query) / len(SCQ_values))
    return avgscq

def MaxSCQ(SCQ_values):
    maxscq=[]
    for query in SCQ_values:
        maxscq.append(max(query))
    return maxscq

def SumSCQ(SCQ_values):
    sumscq=[]
    for query in SCQ_values:
        sumscq.append(sum(query))
    return sumscq


# PMI

In [11]:
from collections import defaultdict
PMI_code = [] 

for doc in code_documents:
    unique_terms = set(doc)
    
    term_co_occurrences_code = defaultdict(int)
    for term1 in unique_terms:
        for term2 in unique_terms:
            if term1 != term2:
                term_co_occurrences_code[(term1, term2)] += 1

    document_PMI = {} 
    for term_pair, co_occurrence_count in term_co_occurrences_code.items():
        term1, term2 = term_pair
        pt1_t2_D = co_occurrence_count / len(doc)
        pt1_D = sum(1 for term in doc if term == term1) / len(doc)
        pt2_D = sum(1 for term in doc if term == term2) / len(doc)
        pt_D = pt1_D * pt2_D

        if pt_D != 0:
            document_PMI[term_pair] = np.log(pt1_t2_D / pt_D)
    PMI_code.append(document_PMI)

PMI_uc = []

for doc in UC_documents:
    unique_terms = set(doc)
    
    term_co_occurrences_uc = defaultdict(int)
    for term1 in unique_terms:
        for term2 in unique_terms:
            if term1 != term2:
                term_co_occurrences_uc[(term1, term2)] += 1

    document_PMI = {} 
    for term_pair, co_occurrence_count in term_co_occurrences_uc.items():
        term1, term2 = term_pair
        pt1_t2_D = co_occurrence_count / len(doc)
        pt1_D = sum(1 for term in doc if term == term1) / len(doc)
        pt2_D = sum(1 for term in doc if term == term2) / len(doc)
        pt_D = pt1_D * pt2_D

        if pt_D != 0:
            document_PMI[term_pair] = np.log(pt1_t2_D / pt_D)
    PMI_uc.append(document_PMI)


def AvgPMI(PMI_values):
    avgpmi=[]
    for query in PMI_values:
        num_terms = len(query)
        if num_terms > 1:
            avgpmi.append(2 * sum(query.values()) * math.exp(math.lgamma(num_terms - 1) - math.lgamma(num_terms)))
        else:
            avgpmi.append(0) 
    return avgpmi

def MaxPMI(PMI_values):
    maxpmi=[]
    for query in PMI_values:
        if query:
            maxpmi.append(max(query.values()))
        else:
            maxpmi.append(0)
    return maxpmi

# QS

In [12]:
#The percentage of documents in the collection containing at least one of the query terms

def QS(documents):
    QSdocs = []
    total_documents = len(documents)
    for query in documents:
        term_occurrence = set()  
        for term in query:
            for doc in documents:
                if term in doc:
                    if query !=doc:
                      term_occurrence.add(doc)
        
        percentage = (len(term_occurrence) / total_documents) * 100
        QSdocs.append(percentage)
    return QSdocs

# CS

In [13]:
def CS(documents, tf_matrix):
    CSdocs = []
    for query in documents:
        term_similarities = []
        for term in query:
            term_docs = [doc for doc in documents if term in doc]
            num_term_docs = len(term_docs)
            if num_term_docs > 1: 
                similarity_sum = 0
                pairs_count = 0
                for i, doc1 in enumerate(term_docs):
                    for j, doc2 in enumerate(term_docs):
                        if i != j:
                            doc1_index = documents.index(doc1)
                            doc2_index = documents.index(doc2)
                            similarity = cosine_similarity(tf_matrix[doc1_index], tf_matrix[doc2_index])
                            similarity_sum += similarity
                            pairs_count += 1
                
                if pairs_count > 0:
                    term_similarities.append(similarity_sum / pairs_count)
        
        if term_similarities:
            CSdocs.append(np.mean(term_similarities))
        else:
            CSdocs.append(0)
    
    return CSdocs


In [16]:

# IDF
avg_idf = AvgIDF(idf_uc_q)
print("Average IDF for UC documents:", avg_idf)

max_idf = MaxIDF(idf_uc_q)
print("Max IDF for UC documents:", max_idf)

dev_idf = DevIDF(idf_uc_q)
print("Deviation of IDF for UC documents:", dev_idf)

# ICTF
avg_ictf = AvgICTF(ictf_uc_q)
print("Average ICTF for UC documents:", avg_ictf)

max_ictf = MaxICTF(ictf_uc_q)
print("Max ICTF for UC documents:", max_ictf)

dev_ictf = DevICTF(ictf_uc_q)
print("Deviation of ICTF for UC documents:", dev_ictf)

# Entropy
avg_entropy = AvgEntropy(entropy_uc)
print("Average Entropy for UC documents:", avg_entropy)

max_entropy = MaxEntropy(entropy_uc)
print("Max Entropy for UC documents:", max_entropy)

med_entropy = MedEntropy(entropy_uc)
print("Median Entropy for UC documents:", med_entropy)

dev_entropy = DevEntropy(entropy_uc)
print("Deviation of Entropy for UC documents:", dev_entropy)

# Variance
avg_variance = AvgVariance(variance_uc)
print("Average Variance for UC documents:", avg_variance)

max_variance = MaxVariance(variance_uc)
print("Max Variance for UC documents:", max_variance)

sum_variance = SumVariance(variance_uc)
print("Sum of Variance for UC documents:", sum_variance)

# SCQ
avg_scq = AvgSCQ(SCQ_uc)
print("Average SCQ for UC documents:", avg_scq)

max_scq = MaxSCQ(SCQ_uc)
print("Max SCQ for UC documents:", max_scq)

sum_scq = SumSCQ(SCQ_uc)
print("Sum of SCQ for UC documents:", sum_scq)

# PMI
avg_pmi = AvgPMI(PMI_uc)
print("Average PMI for UC documents:", avg_pmi)

max_pmi = MaxPMI(PMI_uc)
print("Max PMI for UC documents:", max_pmi)

# QS
qs_uc = QS(UC_documents)
print("QS for UC documents:", qs_uc)

# # CS
# cs_uc = CS(UC_documents, tf_matrix_uc)
# print("CS for UC documents:", cs_uc)


# IDF
avg_idf = AvgIDF(idf_code_q)
print("Average IDF for Code documents:", avg_idf)

max_idf = MaxIDF(idf_code_q)
print("Max IDF for Code documents:", max_idf)

dev_idf = DevIDF(idf_code_q)
print("Deviation of IDF for Code documents:", dev_idf)

# ICTF
avg_ictf = AvgICTF(ictf_code_q)
print("Average ICTF for Code documents:", avg_ictf)

max_ictf = MaxICTF(ictf_code_q)
print("Max ICTF for Code documents:", max_ictf)

dev_ictf = DevICTF(ictf_code_q)
print("Deviation of ICTF for Code documents:", dev_ictf)

# Entropy
avg_entropy = AvgEntropy(entropy_code)
print("Average Entropy for Code documents:", avg_entropy)

max_entropy = MaxEntropy(entropy_code)
print("Max Entropy for Code documents:", max_entropy)

med_entropy = MedEntropy(entropy_code)
print("Median Entropy for Code documents:", med_entropy)

dev_entropy = DevEntropy(entropy_code)
print("Deviation of Entropy for Code documents:", dev_entropy)

# Variance
avg_variance = AvgVariance(variance_code)
print("Average Variance for Code documents:", avg_variance)

max_variance = MaxVariance(variance_code)
print("Max Variance for Code documents:", max_variance)

sum_variance = SumVariance(variance_code)
print("Sum of Variance for Code documents:", sum_variance)

# SCQ
avg_scq = AvgSCQ(SCQ_code)
print("Average SCQ for Code documents:", avg_scq)

max_scq = MaxSCQ(SCQ_code)
print("Max SCQ for Code documents:", max_scq)

sum_scq = SumSCQ(SCQ_code)
print("Sum of SCQ for Code documents:", sum_scq)

# PMI
avg_pmi = AvgPMI(PMI_code)
print("Average PMI for Code documents:", avg_pmi)

max_pmi = MaxPMI(PMI_code)
print("Max PMI for Code documents:", max_pmi)

# QS
qs_code = QS(code_documents)
print("QS for Code documents:", qs_code)

# # CS
# cs_code = CS(code_documents, tf_matrix_code)
# print("CS for Code documents:", cs_code)




Average IDF for UC documents: [2.007855290641676, 1.3193414107492056, 1.6400472825766603, 1.3375512785004884, 1.5958380321011585, 1.430825152143825, 2.282956007377599, 1.2833956018369865, 3.1633768449166517, 2.512613881639551, 2.2749624839019553, 2.4936477302875213, 3.758735510929242, 2.023356710065795, 1.5394868149370244, 2.587700616079942, 1.2934836298787162, 2.1452889417927046, 2.7518507881177783, 1.5603662916125096, 1.7285214108555238, 1.5698116760918952, 2.8691488487819923, 2.218205916255514, 1.8145059363610136, 1.4331302246946551, 2.0108428488363304, 1.5898351478600554, 2.1608810318944687, 1.5975326402915044, 1.5345219703477635, 3.636805739941973, 3.2196066726036325, 1.8443619682942944, 2.2575964762816834, 3.1973539917416542, 2.374101815553282, 2.4378848501324284, 2.55562768097038, 1.5063465595440593, 2.240557649086291, 2.2112268559394064, 2.2703406086355007, 1.122303426180838, 1.3116395867346635, 1.3930480333798705, 1.7003537310365255, 1.9021438112183515, 3.6085951546337554, 2.1